# This is a categorical scrapping project
The goal for the project is to create a scrapper that could monitor the products informations in ebay.

It will start from a given search and return all the project it had got.
Then becomes a much intellegent scrapper that can cover the entire site and adjusting to changes.

It will use xpath first, then it will switch to image recognition mode for futher scrapping.

In [ ]:
xpath_search_input_field = r'//*[(@id = "gh-ac")]'
xpath_search_button = r'//*[(@id = "gh-btn")]'

In [ ]:
ebay_url = "https://www.ebay.com"

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os
import logging
from parsel import Selector
from pprint import *
import time
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import datetime
import re
import copy
import pandas as pd
from IPython.display import display, HTML
import psycopg2
import json

In [ ]:
given_serach_input = "Iphone"

In [ ]:
def web_driver_init():
    driver = webdriver.Chrome()
    driver.get(ebay_url)
    return driver

driver = web_driver_init()

In [ ]:
def search_input_insertation(driver):
    try:
        driver.find_element_by_xpath(xpath_search_input_field).send_keys(given_serach_input)
        driver.find_element_by_xpath(xpath_search_button).click()
    except:
        logging.error("Please switch to a better network condition and try again.")

search_input_insertation(driver)

In [ ]:
db = SqliteDatabase("ebay_search_spider_temp.db")
db.connect()

class BaseModel(Model):
    class Meta:
        database = db
        
class itemOfSearchResult(BaseModel):
    product_sponsored = BooleanField()
    
    product_title = CharField(unique=False)
    product_link = CharField(unique=True)
    product_tags = CharField(null=True)
    product_status = CharField(null=True)
    
    product_price_from = CharField(null=True)
    product_price_to = CharField(null=True)
    product_price_off = CharField(null=True)
    
    product_shipping = CharField(null=True)
    product_fast_returns = BooleanField()
    product_estimated_delivery_time = CharField(null=True)
    product_guaranteed_by = CharField(null=True)
    
    product_top_rated_plus = BooleanField()
    product_fast_n_free = BooleanField()
    product_buy_spry = BooleanField()
    
    product_watching_or_sold = CharField(null=True)
    
    product_inventory_information = CharField(null=True)
    
    product_image_url = TextField(null=True)
    
    scrapped_time = TimeField()
    
try:
    db.drop_tables([itemOfSearchResult], safe=True)
except:
    pass
db.create_tables([itemOfSearchResult], safe=True)
db.close()

In [ ]:
xpath_product_list = """//*[contains(concat( " ", @class, " " ), concat( " ", "li", " " ))]"""

xpath_product_sponsored = """//*[contains(concat( " ", @class, " " ), concat( " ", "promoted-lv", " " ))]//span"""

xpath_product_title = """string(//*[contains(concat( " ", @class, " " ), concat( " ", "vip", " " ))]//text())"""
xpath_product_link = """//*[contains(concat( " ", @class, " " ), concat( " ", "vip", " " ))]/@href"""
xpath_product_tags = """//div[@class="lvsubtitle"][1]/text()"""
xpath_product_status = """//div[@class="lvsubtitle"][2]/text()"""

xpath_product_price_from = """//*[contains(concat( " ", @class, " " ), concat( " ", "prRange", " " ))]/text()[1]"""
xpath_product_price_to = """//*[contains(concat( " ", @class, " " ), concat( " ", "prRange", " " ))]/text()[2]"""
xpath_product_price_off = """//*[contains(concat( " ", @class, " " ), concat( " ", "black", " " ))]/text()"""

xpath_product_shipping = """//*[contains(concat( " ", @class, " " ), concat( " ", "ship", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bfsp", " " ))]/text()"""
xpath_product_fast_returns = """//*[contains(concat( " ", @class, " " ), concat( " ", "lvreturns", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bfsp", " " ))]"""
xpath_product_estimated_delivery_time = """//*[contains(concat( " ", @class, " " ), concat( " ", "fnftxt", " " ))]//b/text()"""
xpath_product_guaranteed_by = """//*[(@id = "item1c58f19dc6")]//b/text()"""

xpath_product_top_rated_plus = """//*[contains(concat( " ", @class, " " ), concat( " ", "iconETRS2", " " ))]"""
xpath_product_fast_n_free = """//*[contains(concat( " ", @class, " " ), concat( " ", "fnf-green", " " ))]"""
xpath_product_buy_spry = """//*[contains(@class, "lvdetails")]//a[contains(@href, "BuySPRY")]"""

xpath_product_watching_or_sold = """//*[contains(concat( " ", @class, " " ), concat( " ", "red", " " ))]/text()"""

xpath_product_inventory_information = """//*[contains(concat( " ", @class, " " ), concat( " ", "lvoptions", " " ))]//a/text()"""

xpath_product_image_url = """//*[contains(concat( " ", @class, " " ), concat( " ", "img", " " ))]//img/@src"""


xpath_next_page_button = """//*[contains(concat( " ", @class, " " ), concat( " ", "next", " " ))]"""

In [ ]:
def collect_page_data(driver):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        html = driver.page_source

        product_html_list = Selector(text=html).xpath(xpath_product_list).extract()
        for product_html in product_html_list:
            search_result = itemOfSearchResult()

            if Selector(text=product_html).xpath(xpath_product_sponsored).extract_first() is not None:
                search_result.product_sponsored = False
            else:
                search_result.product_sponsored = True
            search_result.product_title = str(Selector(text=product_html).xpath(xpath_product_title).extract_first()).strip()
            search_result.product_link = str(Selector(text=product_html).xpath(xpath_product_link).extract_first()).strip()
            search_result.product_tags = str(Selector(text=product_html).xpath(xpath_product_tags).extract_first()).strip()
            search_result.product_status = str(Selector(text=product_html).xpath(xpath_product_status).extract_first()).strip()

            search_result.product_price_from = str(Selector(text=product_html).xpath(xpath_product_price_from).extract_first()).strip()
            search_result.product_price_to = str(Selector(text=product_html).xpath(xpath_product_price_to).extract_first()).strip()
            search_result.product_price_off = str(str(Selector(text=product_html).xpath(xpath_product_price_off).extract_first())).strip()

            search_result.product_shipping = str(Selector(text=product_html).xpath(xpath_product_shipping).extract_first()).strip()
            if Selector(text=product_html).xpath(xpath_product_fast_returns).extract_first() is None:
                search_result.product_fast_returns = False
            else: 
                search_result.product_fast_returns = True
            search_result.product_estimated_delivery_time = str(Selector(text=product_html).xpath(xpath_product_estimated_delivery_time).extract_first()).strip()
            search_result.product_guaranteed_by = str(Selector(text=product_html).xpath(xpath_product_guaranteed_by).extract_first()).strip()

            if Selector(text=product_html).xpath(xpath_product_top_rated_plus).extract_first() is None:
                search_result.product_top_rated_plus = False
            else:
                search_result.product_top_rated_plus = True
            if Selector(text=product_html).xpath(xpath_product_fast_n_free).extract_first() is None:
                search_result.product_fast_n_free = False
            else:
                search_result.product_fast_n_free = True
            if Selector(text=product_html).xpath(xpath_product_buy_spry).extract_first() is None:
                search_result.product_buy_spry = False
            else:
                search_result.product_buy_spry = True

            search_result.product_watching_or_sold = str(Selector(text=product_html).xpath(xpath_product_watching_or_sold).extract_first()).strip()

            search_result.product_inventory_infomation = str(Selector(text=product_html).xpath(xpath_product_inventory_information).extract_first()).strip()

            search_result.product_image_url = str(Selector(text=product_html).xpath(xpath_product_image_url).extract_first()).strip()

            search_result.scrapped_time = str(datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y"))

            search_result.save()
    except Exception as e:
        logging.error(e)
        logging.error("data parsing error: you may need to update the script")

In [ ]:
def get_to_next_page(driver):
    try:
        driver.find_element_by_xpath(xpath_next_page_button).click()
    except:
        raise Exception("All Page are scrapped")
try:
    while True:
        collect_page_data(driver)
        get_to_next_page(driver)
        print("please wait the script while collecting, Successfully Get a new page. ")
        break
except Exception as e:
    logging.error(e)
    print("All pages are scrapped. Script Finished!")

In [ ]:
cursor = None
try: 
    connection = psycopg2.connect("dbname='categorical_keywords' user='postgres' host='mmp' password='mmp' port='5432'")
    connection.autocommit = True
    cursor = connection.cursor()
except:
    pprint("Cannot connect to server")

In [ ]:
# insert in to commercial_goods keywords database
for result in itemOfSearchResult.select():
    try:
        insert_row_command = \
        """
        INSERT INTO public.commercial_goods(
            good_title, good_keys)
            VALUES (%s, %s);
        """
        insert_values = (result.product_title, json.dumps(result.product_title.split()))
        cursor.execute(insert_row_command, insert_values)
        print("Suceessfully inserted a new title")
    except Exception as e:
        logging.error(e)
        print("falied to insert a new title, may be duplicated.") 

In [ ]:
cursor.close()
print("commercial keyword database updated. ")

In [ ]:
# if you want to speed up your transaction, make sure that your transaction will now raise exception first.
# # insert into eaby_scrapping database
# try:
# insert_row_command = \
# """INSERT INTO public.tech_products(
#     search_string, product_title, product_sponsored, product_tags, product_status, product_price_from, product_price_to, product_price_off, product_shipping, product_fast_returns, product_estimated_delivery_time, product_guaranteed_by, product_top_rated_plus, product_fast_n_free, product_buy_spry, product_watching_or_sold, product_inventory_information, product_image_url, scrapped_time, product_link)
#     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

# args_list = []
# for result in itemOfSearchResult.select():
#     insert_values = (given_serach_input,search_result.product_title,search_result.product_sponsored,search_result.product_tags,search_result.product_status,search_result.product_price_from,search_result.product_price_to,search_result.product_price_off,search_result.product_shipping,search_result.product_fast_returns,search_result.product_estimated_delivery_time,search_result.product_guaranteed_by,search_result.product_top_rated_plus,search_result.product_fast_n_free,search_result.product_buy_spry,search_result.product_watching_or_sold,search_result.product_inventory_information,search_result.product_image_url,search_result.scrapped_time,search_result.product_link)
#     args_list.append(copy.deepcopy(insert_values))
    
# psycopg2.extras.execute_values(cursor, insert_row_command, args_list, template=None, page_size=100)

In [21]:
cursor = None
try: 
    connection = psycopg2.connect("dbname='scrapping_ebay' user='postgres' host='mmp' password='mmp' port='5432'")
    connection.autocommit = True
    cursor = connection.cursor()
except:
    pprint("Cannot connect to server")

In [25]:
for search_result in itemOfSearchResult.select():
    try:
        update_row_command = \
        """
        UPDATE public.tech_products
            SET search_string=%s, product_title=%s, product_sponsored=%s, product_tags=%s, product_status=%s, product_price_from=%s, product_price_to=%s, product_price_off=%s, product_shipping=%s, product_fast_returns=%s, product_estimated_delivery_time=%s, product_guaranteed_by=%s, product_top_rated_plus=%s, product_fast_n_free=%s, product_buy_spry=%s, product_watching_or_sold=%s, product_inventory_information=%s, product_image_url=%s, scrapped_time=%s, product_link=%s
            WHERE product_link=%s;
        """
        update_values = (given_serach_input,search_result.product_title,search_result.product_sponsored,search_result.product_tags,search_result.product_status,search_result.product_price_from,search_result.product_price_to,search_result.product_price_off,search_result.product_shipping,search_result.product_fast_returns,search_result.product_estimated_delivery_time,search_result.product_guaranteed_by,search_result.product_top_rated_plus,search_result.product_fast_n_free,search_result.product_buy_spry,search_result.product_watching_or_sold,search_result.product_inventory_information,search_result.product_image_url,search_result.scrapped_time,search_result.product_link,search_result.product_link)
        cursor.execute(update_row_command, update_values)
        print("successfully updated an old product entry")
    except Exception as e:
        logging.error(e)
        print("falied to update an old product entry, this entry maybe a brand new entry. ")

successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old product entry
successfully updated an old produc

In [24]:
for search_result in itemOfSearchResult.select():
    try:
        insert_row_command = \
        """INSERT INTO public.tech_products(
            search_string, product_title, product_sponsored, product_tags, product_status, product_price_from, product_price_to, product_price_off, product_shipping, product_fast_returns, product_estimated_delivery_time, product_guaranteed_by, product_top_rated_plus, product_fast_n_free, product_buy_spry, product_watching_or_sold, product_inventory_information, product_image_url, scrapped_time, product_link)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        insert_values = (given_serach_input,search_result.product_title,search_result.product_sponsored,search_result.product_tags,search_result.product_status,search_result.product_price_from,search_result.product_price_to,search_result.product_price_off,search_result.product_shipping,search_result.product_fast_returns,search_result.product_estimated_delivery_time,search_result.product_guaranteed_by,search_result.product_top_rated_plus,search_result.product_fast_n_free,search_result.product_buy_spry,search_result.product_watching_or_sold,search_result.product_inventory_information,search_result.product_image_url,search_result.scrapped_time,search_result.product_link)
        cursor.execute(insert_row_command, insert_values)
        print("successfully inserted a new product entry. ")
    except Exception as e:
        logging.error(e)
        print("falied to insert a new product entry, may be duplicated.")

successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new prod

ERROR:root:duplicate key value violates unique constraint "tech_products_pkey"
DETAIL:  Key (product_title, product_price_from)=(Apple iPhone7PLUS/7 32GB/256GB/128, None) already exists.



falied to insert a new product entry, may be duplicated.
successfully inserted a new product entry. 
successfully inserted a new product entry. 


ERROR:root:duplicate key value violates unique constraint "tech_products_pkey"
DETAIL:  Key (product_title, product_price_from)=(New listing, None) already exists.



falied to insert a new product entry, may be duplicated.
successfully inserted a new product entry. 
successfully inserted a new product entry. 


ERROR:root:duplicate key value violates unique constraint "tech_products_pkey"
DETAIL:  Key (product_title, product_price_from)=(New listing, None) already exists.



falied to insert a new product entry, may be duplicated.
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 
successfully inserted a new product entry. 


ERROR:root:duplicate key value violates unique constraint "tech_products_pkey"
DETAIL:  Key (product_title, product_price_from)=(New listing, None) already exists.



falied to insert a new product entry, may be duplicated.
successfully inserted a new product entry. 
successfully inserted a new product entry. 


In [ ]:
cursor.close()
print("job finished with ", len(itemOfSearchResult.select(), " product scrapped and uploaded."))